In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

silicon = Material(Materials.get_by_name_first_match('Silicon'))



In [ ]:
BOND_LENGTH = 1.46
PASSIVANT = "H"

In [ ]:
from utils.visualize import visualize_materials
visualize_materials(silicon)

## Create a silicon nanowire

In [ ]:
from mat3ra.made.tools.modify import filter_by_cylinder, rotate, filter_by_box, add_vacuum_sides
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration


slab_config = SlabConfiguration(bulk=silicon, miller_indices=(1,1,0), thickness=2, vacuum=0, use_orthogonal_z=True, xy_supercell_matrix=[[3,0],[0,2]]) 
silicon_slab = create_slab(slab_config)

visualize_materials(silicon_slab)
visualize_materials(silicon_slab, rotation="-90x")

rotated_slab = rotate(silicon_slab, "z", -45)
silicon_nanowire = filter_by_box(rotated_slab, [0.2,0.28,0], [0.78,0.87,1])
visualize_materials(silicon_nanowire)
visualize_materials(silicon_nanowire, rotation="-90x")
nanowire_xy_pbc = rotate(silicon_nanowire, "z", -45)
# nanowire_xy = rotate(nanowire_xy, "x", 90, rotate_cell=True)
nanowire_xy = add_vacuum_sides(nanowire_xy_pbc, 10.0, on_x=True, on_y=True)
visualize_materials(nanowire_xy)
visualize_materials(nanowire_xy, rotation="-90x")

## Add passivation

In [ ]:
from mat3ra.made.tools.modify import add_vacuum
from mat3ra.made.tools.build.utils import merge_materials
from mat3ra.made.tools.build.passivation import PassivationConfiguration, create_passivation, SurfacePassivationBuilder, SurfacePassivationBuilderParameters, CoordinationBasedPassivationBuilderParameters, CoordinationBasedPassivationBuilder

# Surface passivation
nanowire_xy = add_vacuum(nanowire_xy, 10.0, on_top=True, to_bottom=True)
params = SurfacePassivationBuilderParameters(
    shadowing_radius=2.9,
    bond_length=1.46
)
passivation_builder = SurfacePassivationBuilder(params)

# Under-coordination passivation
# params = CoordinationBasedPassivationBuilderParameters(shadowing_radius=1.8, coordination_threshold=4, depth=10.0)
# passivation_builder = CoordinationBasedPassivationBuilder(params)

ANGLE = 90
ANGLE_Z = 45
nanowire_yz = rotate(nanowire_xy, "z", ANGLE_Z)
nanowire_yz= rotate(nanowire_yz, "y", ANGLE)

nanowire_xz = rotate(nanowire_xy, "z", ANGLE_Z)
nanowire_xz =   rotate(nanowire_xz, "x", ANGLE)

# visualize_materials([nanowire_yz, nanowire_xz])
# visualize_materials([nanowire_yz, nanowire_xz], rotation="-90x")

passivation_config_yz = PassivationConfiguration(
    slab=nanowire_yz,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH
)

passivation_config_xz = PassivationConfiguration(
    slab=nanowire_xz,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH,
)

passivated_nanowire_yz = create_passivation(passivation_config_yz, passivation_builder)
visualize_materials([{"material": passivated_nanowire_yz}, {"material": passivated_nanowire_yz, "rotation": "-90x"}])

passivated_nanowire_xz = create_passivation(passivation_config_xz, passivation_builder)
visualize_materials([{"material": passivated_nanowire_xz}, {"material": passivated_nanowire_xz, "rotation": "-90x"}])

reversed_passivated_nanowire_yz = rotate(passivated_nanowire_yz, "y", -ANGLE)
reversed_passivated_nanowire_yz = rotate(reversed_passivated_nanowire_yz, "z", -ANGLE_Z)
reversed_passivated_nanowire_xz = rotate(passivated_nanowire_xz, "x", -ANGLE)
reversed_passivated_nanowire_xz = rotate(reversed_passivated_nanowire_xz, "z", -ANGLE_Z)

passivated_nanowire = merge_materials([reversed_passivated_nanowire_xz, reversed_passivated_nanowire_yz])
visualize_materials([{ "material": passivated_nanowire}, {"material": passivated_nanowire, "rotation": "-90x"}])


In [ ]:
from utils.jupyterlite import set_materials
passivated_nanowire.name = "H-Passivated Silicon Nanowire"
set_materials([passivated_nanowire])